In [1]:
import dynamiqs as dq
import jax.numpy as jnp
from qutip import Qobj, fidelity, rand_dm, fock_dm
import numpy as np
import src.TK_basics as tk
import src.helper_functions as hf
import importlib
importlib.reload(tk)
importlib.reload(hf)
from src.TK_basics import PSD_rho
from src.helper_functions import rho2vec, trunc, linspace
from sklearn.linear_model import RidgeClassifier, LogisticRegression
import time
import matplotlib
from matplotlib import pyplot as plt
matplotlib.use('Qt5Agg') 
from tqdm import tqdm
import pickle
from bisect import bisect_left
import qutip

In [2]:
import src.encoding.classify_sim as csim
import src.encoding.classify_exp as cexp
import importlib
importlib.reload(csim)
importlib.reload(cexp)
from src.encoding.classify_sim import ClassifySim
from src.encoding.classify_exp import ClassifyExp

# Simulation of 4000 training images and 1000 testing images

In [4]:
cs = ClassifySim(compression=16, t_total=17.5, pc_imag=0.175, pq_imag=0.175,
                 dm=10, intervals=3).evolve(4000, 1000, c_ops=True, shuffle=True)

# compression refers to number of pixels in image
# t_total is total time of evolution in microseconds
# pc_imag, pq_imag are the (imaginary) qubit and cavity drives
# dm is dimensions of hilbert space

|██████████| 100.0% ◆ total 04:41 ◆ remaining 00:00


In [7]:
cs.wig(9, scale=1.5, step=0.1) #qutip wigner function for 9 images

In [ ]:
# TODO: 12us and 0.26 as drive works

In [13]:
cs.score("obs", init=True)

method=obs, init=True:  0.855


In [15]:
cs.mle().score("final", init=True)

Applying mle (simulation)...


100%|██████████| 1000/1000 [00:23<00:00, 43.06it/s]


method=final, init=True:  0.832


In [3]:
c2 = ClassifySim(compression=16, t_total=17.5, pc_real=0, pc_imag=0.175, pq_real=0, pq_imag=0.175,
             cdim=15, dm=10, intervals=3).evolve(4000,1000, shuffle=True
             ).addError(10000).score("obs", init=False, rscore=False)

Evolving states (simulation)...


|██████████| 100.0% ◆ total 00:32 ◆ remaining 00:00


method=obs, init=False:  0.844


In [ ]:
c2.mle().score("final", init=False)

Applying mle (simulation)...


 71%|███████   | 2830/4000 [03:02<01:37, 11.98it/s]

In [ ]:
c2.score("final", init=True)

# FOR EXPERIMENT

In [17]:
import src.encoding.classify_sim as csim
import src.encoding.classify_exp as cexp
import importlib
importlib.reload(csim)
importlib.reload(cexp)
from src.encoding.classify_sim import ClassifySim
from src.encoding.classify_exp import ClassifyExp

In [18]:
ce = ClassifyExp().setData(4000, 1000)

In [ ]:
ced = ce.getDrives()

In [ ]:
'''GENERATING NPZ FILES FOR EXPERIMENT'''
for i in range(len(ced[0])):
    dt=1
    QubitI, QubitQ, CavityI, CavityQ = ced[0][i]
    np.savez(f"..\\..\\data\\exp_data\\drives\\train\\train{i}", dt=dt, CavityQ=CavityQ, CavityI=CavityI, QubitQ=QubitQ, QubitI=QubitI)
'''GENERATING NPZ FILES FOR EXPERIMENT'''
for i in range(len(ced[1])):
    dt=1
    QubitI, QubitQ, CavityI, CavityQ = ced[1][i]
    np.savez(f"..\\..\\data\\exp_data\\drives\\test\\test{i}", dt=dt, CavityQ=CavityQ, CavityI=CavityI, QubitQ=QubitQ, QubitI=QubitI)

In [19]:
ce.setObs().mle().getFidelity()

Appltying mle (experiment)...


100%|██████████| 1000/1000 [00:17<00:00, 58.60it/s]
|███▎      |  33.1% ◆ total 02:50 ◆ remaining 05:03

RuntimeError: An internal JAX error interrupted the execution, please report this to the dynamiqs developers by opening an issue on GitHub or sending a message on dynamiqs Slack (links available at https://www.dynamiqs.org/getting_started/lets-talk.html).